# Preprocessing #2

In [155]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, Normalizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn import set_config
set_config(transform_output='pandas')

from category_encoders import OrdinalEncoder


In [156]:
df = pd.read_csv('../data/train.csv')

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [159]:
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
categorical_columns

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [160]:
numeric_columns = df.select_dtypes(include=['int64']).columns.tolist()
numeric_columns = pd.DataFrame(numeric_columns)

In [161]:
numeric_columns = numeric_columns.drop(index=[34])
numeric_columns = numeric_columns.values.flatten().tolist()

In [162]:
df_drop_cat = df.drop(columns=categorical_columns)

In [163]:
ohe = OneHotEncoder(sparse_output=False, drop='if_binary', handle_unknown='ignore')
scaler = StandardScaler()

In [164]:
preprocessor = ColumnTransformer([
  # ('ohe',ohe,categorical_columns),
  ('scaler',scaler,numeric_columns)
])

In [165]:
X = df.drop(columns='SalePrice')
y = df['SalePrice'].copy()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [166]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                 ['Id', 'MSSubClass', 'LotArea', 'OverallQual',
                                  'OverallCond', 'YearBuilt', 'YearRemodAdd',
                                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF',
                                  'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
                                  'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',
                                  'BsmtHalfBath', 'FullBath', 'HalfBath',
                                  'BedroomAbvGr', 'KitchenAbvGr',
                                  'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',
                                  'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
                                  'EnclosedPorch', '3SsnPorch', 'ScreenPorch', ...])])

In [167]:
X_train_proc = preprocessor.transform(X_train)
X_val_proc = preprocessor.transform(X_val)

In [168]:
from sklearn.metrics import (mean_absolute_error, mean_squared_error, root_mean_squared_error,
r2_score, mean_absolute_percentage_error)

def adj_r2(r2, x):
  n = x.shape[0]
  p = x.shape[1]
  return 1 - (((n-1) / (n - p- 1)) * (1 - r2))

def evaluate_regression(model, X, y, name='model'):
  y_pred = model.predict(X)
  MAPE = mean_absolute_percentage_error(y, y_pred)
  r2 = r2_score(y, y_pred)
  RMSE = root_mean_squared_error(y, y_pred)
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  a_r2 = adj_r2(r2, X)
  metrics = ['MAE','MSE','RMSE','MAPE','R2','adj_r2']
  results = pd.DataFrame(columns=metrics, index=[name])
  results['MAE'] = [MAE]
  results['MSE'] = [MSE]
  results['RMSE'] = RMSE
  results['MAPE'] = MAPE
  results['R2'] = r2
  results['adj_r2'] = a_r2

  return results

### Baseline Linear Regression, dropped all categorical and nulls

In [169]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(X_train_proc, y_train)

linear_results_train = evaluate_regression(lr, X_train_proc, y_train, 'baselineLR')
linear_results_val = evaluate_regression(lr, X_val_proc, y_val, 'baselineLR')

In [170]:
linear_results_train

,MAE,MSE,RMSE,MAPE,R2,adj_r2
baselineLR,21063.868333,1.171665e+09,34229.594331,0.124331,0.803562,0.797667


In [171]:
linear_results_val

,MAE,MSE,RMSE,MAPE,R2,adj_r2
baselineLR,22959.21086,1.393347e+09,37327.559394,0.134704,0.818346,0.794314


In [173]:
val_preds = lr.predict(X_val_proc)
pred_log = np.log(val_preds)
y_log = np.log(y_val)
root_mean_squared_error(y_log, pred_log)

0.19807176545303232